In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
data = np.load('../data/metr_la_new.npz')

In [3]:
data['edges']

array([[  0,  37],
       [  0,  54],
       [  0, 116],
       ...,
       [206, 155],
       [206, 159],
       [206, 163]], dtype=int32)

In [4]:
dataset = data['targets']
dataset_size = len(dataset)

In [5]:
for i in range(0, len(dataset)):
    for j in range(0, len(dataset[i])):
        if np.isnan(dataset[i][j]):
            dataset[i][j] = dataset[max(i - 1, 0)][j]

In [6]:
import torch 
import random
from torch.utils.data import TensorDataset, DataLoader


In [7]:
device = "cpu" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [8]:
train_size = int(0.6 * dataset_size)
test_size =  int(0.2 * dataset_size)
vertice = 0
dataset_for_vertice = dataset[:, vertice]

In [9]:
coef = 20
pred_cnt = 12

In [11]:
samples_train = [i for i in random.sample(range(coef, int(dataset_size * 0.7)), train_size)]

X_train = [dataset_for_vertice[i - coef: i] for i in samples_train]
X_train = torch.tensor(X_train).to(device)

y_train = [dataset_for_vertice[i:i + pred_cnt] for i in samples_train]
y_train = torch.tensor(y_train).to(device)

samples_test = [i for i in random.sample(range(int(dataset_size * 0.7), dataset_size - pred_cnt), test_size)]
X_test = [dataset_for_vertice[i - coef: i] for i in samples_test]
X_test = torch.tensor(X_test).to(device)

y_test = [dataset_for_vertice[i:i + pred_cnt] for i in samples_test]
y_test = torch.tensor(y_test).to(device)

/tmp/ipykernel_3666/2937955727.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X_train = torch.tensor(X_train).to(device)


In [12]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(y_train)
print(y_test)


torch.Size([20563, 20]) torch.Size([20563, 12])
torch.Size([6854, 20]) torch.Size([6854, 12])
tensor([[60.2500, 59.7500, 55.4444,  ..., 63.5000, 64.3750, 65.5000],
        [62.8750, 64.2222, 64.5000,  ..., 52.5556, 63.1250, 66.1250],
        [68.4444, 67.8750, 65.1250,  ..., 67.3750, 65.5000, 68.4444],
        ...,
        [67.3750, 68.0000, 67.0000,  ..., 68.5556, 68.7500, 69.5556],
        [62.5556, 64.1250, 66.2500,  ..., 64.7500, 67.5000, 65.7778],
        [65.8750, 65.8889, 63.0000,  ..., 64.5556, 66.5556, 64.7500]])
tensor([[67.8889, 67.7500, 66.7778,  ..., 68.1111, 68.2500, 67.6250],
        [66.3750, 69.2500, 66.1111,  ..., 66.2500, 67.3333, 65.8750],
        [66.3333, 45.7500, 29.7143,  ..., 19.1111, 18.1250, 17.3750],
        ...,
        [63.2500, 62.6667, 64.7143,  ..., 62.3750, 65.7500, 64.6667],
        [67.5000, 67.5000, 67.5000,  ..., 67.5000, 67.5000, 67.5000],
        [67.5000, 67.5000, 67.5000,  ..., 67.5000, 67.5000, 67.5000]])


In [13]:
train_data_set = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_data_set,
                        shuffle = True,
                        batch_size = 8)

test_data_set = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_data_set,
                        shuffle = True,
                        batch_size = 1)

In [14]:
from torch import nn

model_1 = nn.Linear(coef, 1)
model_1.to(device)

model_1 = torch.compile(model_1)


In [15]:
optimizer = torch.optim.Adam(params = model_1.parameters(), lr = 1e-7, weight_decay=1e-5)
loss_fn = nn.L1Loss()

In [16]:
from tqdm import tqdm

In [17]:
torch.set_float32_matmul_precision('high')

In [18]:
def make_a_guess(model, X):
    ans = []
    updated_X = X
    for k in range(pred_cnt):
        ans.append(model(updated_X).to(device))
        updated_X = updated_X[:, :-1]
        updated_X = torch.cat([updated_X, ans[-1]], dim = 1)
    ans = torch.stack(ans, dim = 0).to(device)
    ans = torch.permute(ans, (1, 0, 2)).squeeze(dim = 2)
    return ans

In [19]:
epochs = 100


losses = []

for i in tqdm(range(epochs)):
    for x, y in train_dataloader:
        y_pred = make_a_guess(model = model_1, X=x)
        loss = loss_fn(y_pred, y).to(device)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


100%|██████████| 100/100 [09:02<00:00,  5.42s/it]


In [20]:
px.line(y = np.array(losses))

In [21]:
np.array(losses).mean()

69.5237351027837

In [22]:
model_1.eval()
test_losses_using_model = []
with torch.inference_mode():
    for x, y in test_dataloader:
        y_pred = make_a_guess(model = model_1, X=x)
        loss = loss_fn(y_pred, y).to(device)
        test_losses_using_model.append(loss.item())

mean_loss_using_model = np.array(test_losses_using_model).mean()
mean_loss_using_model

56.34433264491646

In [23]:
type(dataset_for_vertice)

numpy.ndarray

In [24]:
average_val = torch.tensor(dataset_for_vertice.mean())
test_losses_using_average = []

for x, y in test_dataloader:
    y_pred = torch.stack([average_val] * pred_cnt, dim = 0).unsqueeze(dim = 0).to(device)
    loss = loss_fn(y_pred, y).to(device)
    test_losses_using_average.append(loss.item())


mean_loss_using_average = np.array(test_losses_using_average).mean()
mean_loss_using_average

7.001247019917077

In [25]:
test_losses_using_last = []

for x, y in test_dataloader:
    last_val = x[:, -1]
    y_pred = torch.stack([last_val] * pred_cnt, dim = 0).unsqueeze(dim = 0).squeeze(dim = 2).to(device)
    loss = loss_fn(y_pred, y).to(device)
    test_losses_using_last.append(loss.item())


mean_loss_using_last = np.array(test_losses_using_last).mean()
mean_loss_using_last

3.129279187463848

In [26]:
from random import randint

In [27]:
test_losses_using_week_before = []
week_length = 12 * 24 * 7

for i in range(test_size):
    x = randint(week_length, dataset_for_vertice.shape[0] - 1)
    y = torch.tensor(dataset_for_vertice[x]).to(device)
    y_pred = torch.tensor(dataset_for_vertice[x - week_length]).to(device)
    loss = loss_fn(y_pred, y).to(device)
    test_losses_using_week_before.append(loss.item())

mean_loss_using_week_before = np.array(test_losses_using_week_before).mean()
mean_loss_using_week_before

4.602694672400032

In [28]:
from pandas import DataFrame as df

In [29]:
print("Average error for next hour guesses")
print(f"mean_loss_using_average: {mean_loss_using_average}")
print(f"mean_loss_using_last: {mean_loss_using_last}")
print(f"mean_loss_using_week_before :{mean_loss_using_week_before}")
print(f"mean_loss_using_model: {mean_loss_using_model}")


Average error for next hour guesses
mean_loss_using_average: 7.001247019917077
mean_loss_using_last: 3.129279187463848
mean_loss_using_week_before :4.602694672400032
mean_loss_using_model: 56.34433264491646


In [30]:
comparison = df({"mean_loss_using_average": [mean_loss_using_average.item()],
                 "mean_loss_using_last": [mean_loss_using_last.item()],
                 "mean_loss_using_week_before": [mean_loss_using_week_before.item()],
                 "mean_loss_using_model": [mean_loss_using_model.item()]})
comparison.T

,0
mean_loss_using_average,7.001247
mean_loss_using_last,3.129279
mean_loss_using_week_before,4.602695
mean_loss_using_model,56.344333


In [32]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:0000:0100:01


In [36]:
from catboost import CatBoostRegressor

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: catboost/private/libs/target/data_providers.cpp:639: Currently only multi-regression, multilabel and survival objectives work with multidimensional target